# Training logic

In [1]:
import torch
from torch import nn 
from torch import optim
import torch.nn.functional as F

from torchvision import datasets, transforms

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
batch_size = 32

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('dataset/', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize(mean=(0.5,), std=(0.5,))
                   ])),
    batch_size=batch_size,
    shuffle=True)

In [4]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channel, out_channel):
        super(ResidualBlock, self).__init__()
        
        self.in_channel, self.out_channel = in_channel, out_channel
        
        self.conv1 = nn.Conv2d(in_channel, out_channel, kernel_size=1, padding=0)
        self.conv2 = nn.Conv2d(out_channel, out_channel, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(out_channel, out_channel, kernel_size=1, padding=0)
        
        if in_channel != out_channel:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channel, out_channel, kernel_size=1, padding=0)
            )
        else:
            self.shortcut = nn.Sequential()
    
    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = F.relu(self.conv2(out))
        out = F.relu(self.conv3(out))
        out = out + self.shortcut(x)
        return out

class ResNet(nn.Module):
    def __init__(self, color='gray'):
        super(ResNet, self).__init__()
        if color == "gray":
            self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        elif color == "rgb":
            self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
            
        self.resblock1 = ResidualBlock(32, 64)
        self.resblock2 = ResidualBlock(64, 64)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc1 = nn.Linear(64, 64)
        self.fc2 = nn.Linear(64, 10)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = self.resblock1(x)
        x = self.resblock2(x)
        x = self.avgpool(x)
        x = torch.flatten(x,1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)
        return x

In [5]:
model = ResNet().to(device)

###  Learning Rate Scheduler

In [6]:
optimizer = optim.Adam(model.parameters(), lr=0.03)

In [7]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [8]:
scheduler = ReduceLROnPlateau(optimizer, mode = 'min', verbose=True)

In [14]:
def train_loop(dataloader, model, loss_fn, optimizer, scheduler, epoch):
    model.train()
    size = len(dataloader)


    for batch, (x, y) in enumerate(dataloader):
        x, y = x.to(device), y.to(device)

        pred = model(x)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()   # 초기화

        loss.backward()     # 미분값 계산

        optimizer.step

        if batch % 100 == 0:
            loss = loss.item()
            print(f'Epoch {epoch} : [{batch}/{size}] loss : {loss}')

    scheduler.step(loss)

    return loss.item()

In [15]:
for epoch in range(10):
    loss = train_loop(train_loader, model, F.nll_loss, optimizer, scheduler, epoch)
    print(f'epoch : {epoch} loss : {loss}')

Epoch 0 : [0/1875] loss : 2.289947748184204
Epoch 0 : [100/1875] loss : 2.312828779220581
Epoch 0 : [200/1875] loss : 2.3105626106262207
Epoch 0 : [300/1875] loss : 2.3222038745880127
Epoch 0 : [400/1875] loss : 2.3143937587738037
Epoch 0 : [500/1875] loss : 2.3173789978027344
Epoch 0 : [600/1875] loss : 2.3045566082000732
Epoch 0 : [700/1875] loss : 2.333916664123535
Epoch 0 : [800/1875] loss : 2.3437726497650146
Epoch 0 : [900/1875] loss : 2.2667899131774902
Epoch 0 : [1000/1875] loss : 2.3023641109466553
Epoch 0 : [1100/1875] loss : 2.3086001873016357
Epoch 0 : [1200/1875] loss : 2.3219830989837646
Epoch 0 : [1300/1875] loss : 2.3299734592437744
Epoch 0 : [1400/1875] loss : 2.2931113243103027
Epoch 0 : [1500/1875] loss : 2.341890335083008
Epoch 0 : [1600/1875] loss : 2.296038866043091
Epoch 0 : [1700/1875] loss : 2.320039749145508
Epoch 0 : [1800/1875] loss : 2.3274431228637695
epoch : 0 loss : 2.297598123550415
Epoch 1 : [0/1875] loss : 2.2942771911621094
Epoch 1 : [100/1875] loss 